# housekeeping

In [4]:
install.packages('brms')

Installing package into ‘/usr/local/lib/R/site-library’
(as ‘lib’ is unspecified)

also installing the dependencies ‘lazyeval’, ‘numDeriv’, ‘plyr’, ‘httpuv’, ‘fontawesome’, ‘sourcetools’, ‘later’, ‘promises’, ‘htmlwidgets’, ‘miniUI’, ‘crosstalk’, ‘zoo’, ‘igraph’, ‘StanHeaders’, ‘inline’, ‘gridExtra’, ‘RcppParallel’, ‘RcppEigen’, ‘BH’, ‘checkmate’, ‘tensorA’, ‘distributional’, ‘ggridges’, ‘reshape2’, ‘shiny’, ‘colourpicker’, ‘DT’, ‘dygraphs’, ‘gtools’, ‘markdown’, ‘shinyjs’, ‘shinythemes’, ‘threejs’, ‘xtable’, ‘xts’, ‘mvtnorm’, ‘Brobdingnag’, ‘globals’, ‘listenv’, ‘parallelly’, ‘Rcpp’, ‘rstan’, ‘loo’, ‘posterior’, ‘rstantools’, ‘bayesplot’, ‘shinystan’, ‘bridgesampling’, ‘future’, ‘matrixStats’, ‘nleqslv’, ‘coda’, ‘abind’




In [5]:
library(brms)

Loading required package: Rcpp

Loading 'brms' package (version 2.19.0). Useful instructions
can be found by typing help('brms'). A more detailed introduction
to the package is available through vignette('brms_overview').


Attaching package: ‘brms’


The following object is masked from ‘package:stats’:

    ar




In [6]:
library(tidyverse)

Warning message in system("timedatectl", intern = TRUE):
“running command 'timedatectl' had status 1”
── Attaching packages ─────────────────────────────────────── tidyverse 1.3.1 ──

✔ ggplot2 3.4.1     ✔ purrr   1.0.1
✔ tibble  3.2.1     ✔ dplyr   1.1.1
✔ tidyr   1.3.0     ✔ stringr 1.4.1
✔ readr   2.1.4     ✔ forcats 1.0.0

── Conflicts ────────────────────────────────────────── tidyverse_conflicts() ──
✖ dplyr::filter() masks stats::filter()
✖ dplyr::lag()    masks stats::lag()



In [7]:
install.packages('caret')

Installing package into ‘/usr/local/lib/R/site-library’
(as ‘lib’ is unspecified)

also installing the dependencies ‘shape’, ‘future.apply’, ‘progressr’, ‘SQUAREM’, ‘diagram’, ‘lava’, ‘prodlim’, ‘proxy’, ‘iterators’, ‘clock’, ‘gower’, ‘hardhat’, ‘ipred’, ‘timeDate’, ‘e1071’, ‘foreach’, ‘ModelMetrics’, ‘pROC’, ‘recipes’




In [8]:
library(caret)

Loading required package: lattice


Attaching package: ‘caret’


The following object is masked from ‘package:purrr’:

    lift




In [17]:
remotes::install_github("stan-dev/cmdstanr")

isoband  (0.2.6  -> 0.2.7 ) [CRAN]
digest   (0.6.30 -> 0.6.31) [CRAN]
ggplot2  (3.4.1  -> 3.4.2 ) [CRAN]
ps       (1.7.3  -> 1.7.4 ) [CRAN]
jsonlite (1.8.3  -> 1.8.4 ) [CRAN]


Installing 5 packages: isoband, digest, ggplot2, ps, jsonlite

Installing packages into ‘/usr/local/lib/R/site-library’
(as ‘lib’ is unspecified)



── R CMD build ─────────────────────────────────────────────────────────────────
* checking for file ‘/tmp/RtmpP5togP/remotes9cc29de341f/stan-dev-cmdstanr-ab30e07/DESCRIPTION’ ... OK
* preparing ‘cmdstanr’:
* checking DESCRIPTION meta-information ... OK
* checking for LF line-endings in source and make files and shell scripts
* checking for empty or unneeded directories
Omitted ‘LazyData’ from DESCRIPTION
* building ‘cmdstanr_0.5.3.tar.gz’



Installing package into ‘/usr/local/lib/R/site-library’
(as ‘lib’ is unspecified)



In [18]:
library('cmdstanr')

This is cmdstanr version 0.5.3

- CmdStanR documentation and vignettes: mc-stan.org/cmdstanr

- Use set_cmdstan_path() to set the path to CmdStan

- Use install_cmdstan() to install CmdStan



# データ準備

In [9]:
df <- read_csv('./df_cat.csv')

Rows: 581012 Columns: 13
── Column specification ────────────────────────────────────────────────────────
Delimiter: ","
dbl (13): Elevation, Aspect, Slope, Horizontal_Distance_To_Hydrology, Vertic...

ℹ Use `spec()` to retrieve the full column specification for this data.
ℹ Specify the column types or set `show_col_types = FALSE` to quiet this message.


In [10]:
df <- df |> mutate(Wilderness = as.factor(Wilderness_Areas),
             Soil = as_factor(Soil_Types),
             Cover = as.factor(Cover_Type)) |>
    select(-c(Wilderness_Areas, Soil_Types, Cover_Type))

# データ分割

In [11]:
# データ分割 ####
set.seed(1013)
trainIndex <- caret::createDataPartition(df$Cover, p = 0.8, list = FALSE)
training <- df[trainIndex, ]
testing <- df[-trainIndex, ]

# multimodel

In [12]:
colnames(training)

[1] "Elevation"                          "Aspect"                            
 [3] "Slope"                              "Horizontal_Distance_To_Hydrology"  
 [5] "Vertical_Distance_To_Hydrology"     "Horizontal_Distance_To_Roadways"   
 [7] "Hillshade_9am"                      "Hillshade_Noon"                    
 [9] "Hillshade_3pm"                      "Horizontal_Distance_To_Fire_Points"
[11] "Wilderness"                         "Soil"                              
[13] "Cover"

In [31]:
set_cmdstan_path('/usr/local/lib/R/site-library/cmdstanr')

Warning message:
“Can't find CmdStan makefile to detect version number. Path may not point to valid installation.”
CmdStan path set to: /usr/local/lib/R/site-library/cmdstanr



In [32]:
model <- brm(
  data = training,
  family = 'categorical',
  Cover ~ 1 + Horizontal_Distance_To_Fire_Points + (1 + Horizontal_Distance_To_Fire_Points | Wilderness),
  prior = c(
    prior(normal(0, 10), class=Intercept),
    prior(normal(0, 10), class=b)
    ),
  iter = 2000, warmup = 1000, chains = 4, cores = 4, seed = 1013,
  opencl=opencl(c(0, 0)), backend='cmdstanr',
  file = './multimodel'
)

Warning message:
“Specifying global priors for regression coefficients in categorical models is deprecated. Please specify priors separately for each response category.”
Warning message:
“Specifying global priors for regression coefficients in categorical models is deprecated. Please specify priors separately for each response category.”


ERROR: ignored

In [20]:
?set_cmdstan_path